## ADR - target association identification
GNU General Public License v3.0 - Robert Ietswaart

### Citation:
Robert Ietswaart<sup>\*,#</sup>, Seda Arat<sup>\*,#</sup>, Amanda X. Chen<sup>\*</sup>, 
Saman Farahmand<sup>\*</sup>, Bumjun Kim, William DuMouchel, 
Duncan Armstrong, Alexander Fekete, Jeffrey J. Sutherland<sup>#</sup>, Laszlo Urban<sup>#</sup>  
*Machine learning guided association of adverse drug reactions with in vitro target-based 
pharmacology* (2019), [BioRxiv; 750950](https://www.biorxiv.org/content/10.1101/750950v2).

In [1]:
import pandas as pd
import numpy as np
import copy
import os.path
import re
from scipy.stats import ttest_ind, ttest_rel
from statsmodels.stats.multitest import fdrcorrection

In [4]:
path = "~/ADRtarget/data/"
filename = 'mergedassaynumber-to-onewordname.csv'#NB: this file is not publicly available due to legal restrictions from Novartis.
#Sample mapping file to indicate the format:
# filename = 'sample_assayID-to-gene.csv'

targets = pd.read_csv(path+filename)

for i in targets.index:
    A = re.sub('merged_', '', targets['Number'][i])
    targets.loc[i,'Number'] = re.sub('_', '.', A)
    if not str(targets['Type of Assay'][i]) == 'nan':
        targets.loc[i,'Name']=targets['Name'][i]+'.'+targets['Type of Assay'][i]
                 
targets = targets.drop('Type of Assay',axis=1)

### Determine associations for each replicate run individually

In [ ]:
level = 'HLGT'
diff_SEEDS = [112358,2662851,49,5332728]
N_reps = 5

for i in range(N_reps):
    if i == 0:
        path='~/ADRtarget/data/'+level+'/'
    else:
        seed=diff_SEEDS[i-1]
        path='~/ADRtarget/data/'+level+'_diffSeed_'+str(seed)+'/'
    filename='Features_ADRs_predictions.csv'            
    df = pd.read_csv(path+filename,index_col=0)#assay classes are indices
    colnames=list(df.columns)
    N_ADR_terms=len(colnames)

    cl = {'vals':['0-3','3-30','>30']}
    cl = pd.DataFrame(cl)
    N_cat = 3

    output_dict = dict()
    cl_dict = {1:'0-3uM',2:'3-30uM',3:'>30uM'}

    for j in range(1,N_ADR_terms+1):
        cn = colnames[j-1]#NB: ADR list is 1 based, python 0-based 
        print(j,cn)
        if i == 0:
            path = '~/ADRtarget/data/'+level+'/Importance_Gini/'
        else:
            seed = diff_SEEDS[i-1]
            path = '~/ADRtarget/data/'+level+'_diffSeed_'+str(seed)+'/Importance_Gini/'
            
        filename = 'imp_for_ADR-'+str(j)+'_signf.csv'
        if os.path.isfile(path+filename):
            gini = pd.read_csv(path+filename)
            gini = gini.rename(columns={'Unnamed: 0': 'cl', 'x':'gini_score'})

            assay_ID = dict()#get assays in gini in useable format
            for a_cl in gini['cl']:
                temp = re.sub('A', '', a_cl)
                temp = re.sub('CL', '', temp)
                temp = re.split('_',temp)
                a_key = temp[0]
                try:
                    assay_ID[a_key].append(a_cl)
                except KeyError:
                    assay_ID[a_key] = [a_cl]

            for a in assay_ID.keys():
                tar = targets['Name'][targets['Number']==a].values[0]
                if len(assay_ID[a]) > 1:#at least 2 classes were significant: now you can compare assay results
                    imp_cl = []
                    for c in range(1,N_cat+1):#class index 1-based, python 0-based
                        if 'A'+a+'_CL'+str(c) in assay_ID[a]:
                            imp_cl.append(True)
                        else:
                            imp_cl.append(False)

                    if (sum(df.loc[assay_ID[a],cn]) > 0):#if sign classes have non-zero prob (=model outputs)
                        no_corr_test_prob = -1
                        no_corr_test_bool = True
                        #build csv file with same results
                        print('Accepted: Assay:',a,': ',tar,' level',level,'term:',j,cn)
                        try:
                            output_dict['ADR'].append(cn.replace('.',' '))
                            output_dict['target'].append(tar)
                            output_dict['assay'].append(a)
                            for c in range(1,N_cat+1):
                                temp_cl = 'A'+a+'_CL'+str(c)
                                if temp_cl in assay_ID[a]:
                                    #test for no_correlation
                                    if no_corr_test_prob == -1:
                                        no_corr_test_prob = df.loc[temp_cl,cn]
                                    elif no_corr_test_prob == df.loc[temp_cl,cn]:
                                        pass
                                    else:
                                        no_corr_test_prob = df.loc[temp_cl,cn]
                                        no_corr_test_bool = False
                                    output_dict[cl_dict[c]].append(df.loc[temp_cl,cn])
                                else:
                                    output_dict[cl_dict[c]].append(np.nan)
                            output_dict['no_corr'].append(no_corr_test_bool)
                        except KeyError:
                            output_dict['ADR'] = [cn.replace('.',' ')]
                            output_dict['target'] = [tar]
                            output_dict['assay'] = [a]
                            for c in range(1,N_cat+1):
                                temp_cl='A'+a+'_CL'+str(c)
                                if temp_cl in assay_ID[a]:
                                    #test for no_correlation
                                    if no_corr_test_prob == -1:
                                        no_corr_test_prob = df.loc[temp_cl,cn]
                                    elif no_corr_test_prob == df.loc[temp_cl,cn]:
                                        pass
                                    else:
                                        no_corr_test_prob = df.loc[temp_cl,cn]
                                        no_corr_test_bool = False
                                    output_dict[cl_dict[c]] = [df.loc[temp_cl,cn]]
                                else:
                                    output_dict[cl_dict[c]] = [np.nan]
                            output_dict['no_corr'] = [no_corr_test_bool]
                    else:
                        print('Zeros only, Assay:',a,': ',tar,' level',level,'term:',j,cn) 
                else:
                    print('One class significant only, Assay:',a,': ',tar,' level',level,'term:',j,cn)
        else:
            print('Gini file for ADR ',j,cn,' does not exist.', version_model,level)

    if i == 0:
        path = '~/ADRtarget/data/'+level+'/'
        filename = level+'_ADR_target_assoc.csv'
    else:
        seed=diff_SEEDS[i-1]
        path='~/ADRtarget/data/'+level+'_diffSeed_'+str(seed)+'/'
        filename = level+'_ADR_target_assoc_seed'+str(seed)+'.csv'
    
    output_df=pd.DataFrame(output_dict)
    output_df.to_csv(os.path.join(path,filename))

### Reproducibility analysis between 5 replicate runs
identify for which ADRs the model only predicts zeros: exclude those ADRs (below)

In [ ]:
ADR_excl=dict()
for i in range(N_reps):
    filename='summ_metrics_ADRs.csv'
    if i == 0:
        path='~/ADRtarget/data/'+level+'/'
        MetADR = pd.read_csv(path+filename,index_col=0)
    else:
        seed=diff_SEEDS[i-1]
        path='~/ADRtarget/data/'+level+'_diffSeed_'+str(seed)+'/'
        MetADR_OLD=copy.deepcopy(MetADR['ADR'])
        MetADR = pd.read_csv(path+filename,index_col=0)
        MetADR['ADR']=MetADR_OLD
    MetADR_nonan=MetADR[~np.isnan(MetADR['precision'])]
    MetADR_nonan['Pos']=MetADR_nonan['TP']+MetADR_nonan['FN']

    ADR_excl[i]=MetADR[np.isnan(MetADR['precision'])]['ADR']
    ADR_excl[i]=list(ADR_excl[i])
    for j in range(len(ADR_excl[i])):
        ADR_excl[i][j]=re.sub('\(|\)|,|\'|-',' ',ADR_excl[i][j])
    print(len(ADR_excl[i]))


First use meddra sheet to get mapping from HLGT to SOC level for ordering in heatmap

In [ ]:
path='~/ADRtarget/data/'
filename='meddr1a_full_12092006.xlsx'
meddra = pd.read_excel(path+filename)

HLGT2SOC=dict()
for index, arow in meddra.iterrows():
    hlgt = str(arow['HLGT_TXT']).lower()
    hlgt=re.sub('\(|\)|,|\'|-',' ',hlgt)
    try:
        HLGT2SOC[hlgt]
    except KeyError:
        HLGT2SOC[hlgt] = str(arow['SOC_TXT']).lower()

### Load all unfiltered target-ADRs from individual replicates

In [ ]:
AT=dict()
for i in range(N_reps):
    if i == 0:
        path = '~/ADRtarget/data/'+level+'/'
        filename = level+'_ADR_target_assoc.csv'
    else:
        seed=diff_SEEDS[i-1]
        path='~/ADRtarget/data/'+level+'_diffSeed_'+str(seed)+'/'
        filename = level+'_ADR_target_assoc_seed'+str(seed)+'.csv' 
    AT[i]= pd.read_csv(path+filename,index_col=0)

for i in range(N_reps):
    SOC=[]
    for j in AT[i].index:
        SOC.append(HLGT2SOC[AT[i]['ADR'][j]])
    AT[i].insert(loc=1,column='SOC',value=SOC)

### Filtering out ADRs from AT
Filter out ADRs that have no predictive value (ADR_excl)

Also filter out the following SOC classes: 

-general disorders and administration site conditions

-injury, poisoning and procedural complications

-poisoning and procedural complications

-surgical and medical procedures

-neoplasms benign, malignant and unspecified (incl cysts and polyps)

-investigations

-social circumstances

In [ ]:
SOC_excl=['surgical and medical procedures',
        'general disorders and administration site conditions',
        'injury, poisoning and procedural complications',
        'investigations',
        'poisoning and procedural complications',
        'neoplasms benign, malignant and unspecified (incl cysts and polyps)',
        'social circumstances']

In [ ]:
for i in range(N_reps):
    AT[i]=AT[i][~AT[i]['ADR'].isin(ADR_excl[i])]
    AT[i]=AT[i][~AT[i]['SOC'].isin(SOC_excl)]
    AT[i]=AT[i].sort_values(by=['SOC','ADR','target'])
    AT[i]=AT[i].drop(['no_corr'],axis=1)
    AT[i]=AT[i].reindex(columns=['SOC','ADR','target','assay','0-3uM','3-30uM','>30uM'])

### Merge (union = outer) to see overlaps

In [ ]:
ATM = pd.merge(AT[0],AT[1], on=['SOC','ADR','target','assay'],how='outer')
ATM.columns = ['SOC','ADR','target','assay','0:0-3uM','0:3-30uM','0:>30uM','1:0-3uM','1:3-30uM','1:>30uM']
for i in range(2,N_reps):
    ATM= pd.merge(ATM,AT[i], on=['SOC','ADR','target','assay'],how='outer')
    colnames={'0-3uM':str(i)+':0-3uM','3-30uM':str(i)+':3-30uM','>30uM':str(i)+':>30uM'}
    ATM=ATM.rename(columns=colnames) 
    
ATM=ATM.sort_values(by=['SOC','ADR','target'])

## Get summary statistics

In [ ]:
ct={1:[],2:[],3:[]}
cl_dict={1:'0-3uM',2:'3-30uM',3:'>30uM'}
for j in ATM.index:
    for cl in cl_dict.keys():
        count=0
        for i in range(N_reps):
            if not np.isnan(ATM[str(i)+':'+cl_dict[cl]][j]):
                count+=1
        ct[cl].append(count)
ATM.insert(loc=4,column='N:'+cl_dict[1],value=ct[1])
ATM.insert(loc=5,column='N:'+cl_dict[2],value=ct[2])
ATM.insert(loc=6,column='N:'+cl_dict[3],value=ct[3])

#get averages over reps
ct={1:[],2:[],3:[]}
se={1:[],2:[],3:[]}
for j in ATM.index:
    for cl in cl_dict.keys():
        ct[cl].append( \
    np.mean(ATM.loc[j,['0:'+cl_dict[cl],'1:'+cl_dict[cl],'2:'+cl_dict[cl],'3:'+cl_dict[cl],'4:'+cl_dict[cl]]]))
        se[cl].append( \
    np.std(ATM.loc[j,['0:'+cl_dict[cl],'1:'+cl_dict[cl],'2:'+cl_dict[cl],'3:'+cl_dict[cl],'4:'+cl_dict[cl]]]) \
                      / np.sqrt(ATM.loc[j,'N:'+cl_dict[cl]]))    
ATM.insert(loc=7,column='mean:'+cl_dict[1],value=ct[1])
ATM.insert(loc=8,column='mean:'+cl_dict[2],value=ct[2])
ATM.insert(loc=9,column='mean:'+cl_dict[3],value=ct[3])
ATM.insert(loc=10,column='sem:'+cl_dict[1],value=se[1])
ATM.insert(loc=11,column='sem:'+cl_dict[2],value=se[2])
ATM.insert(loc=12,column='sem:'+cl_dict[3],value=se[3])

### Filter out non-significant associations

In [ ]:
alpha_FDR = 0.1
pvals = []
cl_dict = {1:'0-3uM',2:'3-30uM',3:'>30uM'}

#filter out ADR-targets that were only found 1: not reproducible
temp = ATM[['N:'+cl_dict[1],'N:'+cl_dict[2],'N:'+cl_dict[3]]]>1
ATM = ATM[temp.any(axis=1)]

for j in ATM.index:
    count=0
    data_sample1=[]
    data_sample2=[]
    for cl in cl_dict.keys():
        if ATM.loc[j,'N:'+cl_dict[cl]]>1: 
            if count==0:
                data_sample1=list(ATM.loc[j,['0:'+cl_dict[cl],'1:'+cl_dict[cl],'2:'+cl_dict[cl],'3:'+cl_dict[cl],'4:'+cl_dict[cl]]])
                count+=1
            elif count==1:
                data_sample2=list(ATM.loc[j,['0:'+cl_dict[cl],'1:'+cl_dict[cl],'2:'+cl_dict[cl],'3:'+cl_dict[cl],'4:'+cl_dict[cl]]])
    stat,pval=ttest_ind(data_sample1,data_sample2,nan_policy='omit')#independent samples t test

    if np.isnan(stat):#nan data so drop and do not test for significance
        print(stat,pval,'drop',j)
        ATM=ATM.drop(j) 
    else:
        pvals.append(pval)

BOOL, qvals = fdrcorrection(pvals, alpha=1)
ATM.insert(loc=7,column='pval',value=pvals)
ATM.insert(loc=8,column='padj',value=qvals)
ATM = ATM[ATM['padj']<alpha_FDR]

ATM = ATM.sort_values(by=['SOC','ADR','target'])
path = '~/ADRtarget/data/'
filename = level+'_ADR_target_assoc_final.csv'
ATM.to_csv(path+filename)